In [ ]:
import os
import sys
import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, unique, vstack, join, Column

sys.path.insert(0, '..')
from surface_brightness import surface_brightness


In [ ]:
#Input / Output Information
input_data_table = 'Rigault 2015 data table.csv'
fits_directory = '/Volumes/Media RED/fits files/Rigault 15 Supernova' #The directory to look for .fits files in
output_file_name = "Rigault 2015" #The output file name without an extension


In [ ]:
if os.path.exists(fits_directory) and os.listdir(fits_directory):
    cord_dict = {} #for supernova coordinates
    red_dict = {}#for supernova redshifts
    
    rigault_15_table = Table.read(input_data_table, format = 'csv')
    for row in rigault_15_table:
        red_dict[row['sn']] = row['z']
        cord_dict[row['sn']] = SkyCoord(row['RA'] + ' ' + row['Dec'], unit = (u.hourangle, u.deg))
    
    brightness = surface_brightness(cord_dict, red_dict)
    
    print('Calculating NUV values using a 2kpc radius aperture\n', flush = True)
    nuv_2kpc = brightness.create_tables('NUV', fits_directory, 2, print_progress = True)
    
    print('\nCalculating FUV values using a 2kpc radius aperture\n', flush = True)
    fuv_2kpc = brightness.create_tables('FUV', fits_directory, 2, print_progress = True)
    
    #Create plots and save them to file
    print('Creating Plots')
    brightness.create_plots(nuv_2kpc[0], 'NUV', 2)
    brightness.create_plots(fuv_2kpc[0], 'FUV', 2)
    
    #Create a table with the desired output information
    print('Joining Tables\n', flush = True)
    
    #Add columns for AB Magnitude
    NUV_AB = Column(name='NUV AB Magnitude',
                    data = -2.5 * np.log10(nuv_2kpc[0]['Flux'] / 2.06E-16) + 20.08)
    
    NUV_AB_err = Column(name='NUV AB Magnitude Error',
                        data = (2.5 * nuv_2kpc[0]['Flux Error']) / (nuv_2kpc[0]['Flux'] * np.log(10)))
    
    FUV_AB = Column(name='FUV AB Magnitude',
                    data = -2.5 * np.log10(fuv_2kpc[0]['Flux'] / 1.40E-15) + 18.82)
    
    FUV_AB_err = Column(name='FUV AB Magnitude Error',
                        data = (2.5 * fuv_2kpc[0]['Flux Error']) / (fuv_2kpc[0]['Flux'] * np.log(10)))
    
    NUV_AB.unit = u.dimensionless_unscaled
    NUV_AB_err.unit = u.dimensionless_unscaled
    FUV_AB.unit = u.dimensionless_unscaled
    FUV_AB_err.unit = u.dimensionless_unscaled
    
    nuv_2kpc[0].add_column(NUV_AB)
    nuv_2kpc[0].add_column(NUV_AB_err)
    fuv_2kpc[0].add_column(FUV_AB)
    fuv_2kpc[0].add_column(FUV_AB_err)
                           
    nuv_2kpc[0].remove_columns(['Flux', 'Flux Error'])
    fuv_2kpc[0].remove_columns(['Flux', 'Flux Error']) 
    
    #Merge the NUV and FUV tables
    merged_table = join(nuv_2kpc[0], fuv_2kpc[0], join_type = 'outer', keys = ['sn'])
    merged_table.rename_column('NUV 2kpc Exposure Time', 'NUV Exposure Time')
    merged_table.rename_column('FUV 2kpc Exposure Time', 'FUV Exposure Time')
    merged_table.rename_column('Redshift_1', 'Redshift')
    merged_table.rename_column('Redshift Error_1', 'Redshift Error')
    
    for row in merged_table:
        if str(row["Redshift"]) == "--":
            row["Redshift"] = row["Redshift_2"]
            row["Redshift Error"] = row["Redshift Error_2"]
    
    merged_table.remove_columns(["Redshift_2", "Redshift Error_2"])
    
    #Rearrange the column order
    final_table = merged_table['sn', 'Redshift', 'Redshift Error',
                               'NUV Exposure Time',
                               'NUV 2kpc Luminosity',
                               'NUV 2kpc Luminosity Error',
                               'NUV 2kpc Surface Brightness',
                               'NUV 2kpc Surface Brightness Error',
                               'NUV AB Magnitude',
                               'NUV AB Magnitude Error',
                               'FUV Exposure Time',
                               'FUV 2kpc Luminosity',
                               'FUV 2kpc Luminosity Error',
                               'FUV 2kpc Surface Brightness',
                               'FUV 2kpc Surface Brightness Error',
                               'FUV AB Magnitude',
                               'FUV AB Magnitude Error']
    
    merged_log = vstack([nuv_2kpc[1], fuv_2kpc[1]])
    merged_log.sort('File Path')
    
    #Write data to an output file
        #The csv format does not have an 'overwrite' kwarg,
        #but an existing file will be overwritten
    final_table.write(output_file_name + '.fits', overwrite = True, format = 'fits')
    final_table.write(output_file_name + '.csv', format = 'ascii.csv')
    merged_log.write(output_file_name + ' log.csv', format = 'ascii.csv')
    print('Finished')

elif not os.path.exists(fits_directory):
    print('Specified directory of .fits files does not exist')
    
else:
    print('Specified directory of .fits files is empty')